In [316]:
import requests_cache

import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import date, timedelta
import os
import csv
from datetime import datetime
from dateutil import parser

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

In [317]:
requests_cache.install_cache('demo_cache',expire_after=None,allowable_methods=['GET'])
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been 

In [318]:
link = "https://www.worldometers.info/coronavirus/"

driver.get(link)

def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

In [319]:
header = ["Country","Date","TotalCases","NewCases","TotalDeaths","NewDeaths","TotalRecovered",
"NewRecovered","ActiveCases","Serious,Critical","Tot Cases/1M pop","Deaths/1M pop","TotalTests","Tests/1M pop","Population","Continent"]

In [320]:
row_total = driver.find_elements(By.XPATH,"//tr[@class='even'] | //tr[@class='odd']")

rows_past = []
rows_past_modify = []
rows_yesterday = []
rows_two_days_ago = []
count = 0
row_count = 226

for n in row_total:
    n = n.get_attribute('textContent')
    if count == row_count:
        rows_past.append(n.split("\n"))
    else:
        count+=1

for n in rows_past:
    n.pop(0)
    n = n[: len(n) - 6]
    rows_past_modify.append(n)

rows_yesterday,rows_two_days_ago = split_list(rows_past_modify)

rows_yesterday = [ele for ele in rows_yesterday if ele != []]
rows_two_days_ago = [ele for ele in rows_two_days_ago if ele != []]

In [321]:
today = date.today()
yesterday = (today - timedelta(1)).strftime("%d-%m-%Y")
two_days_ago = (today - timedelta(2)).strftime("%d-%m-%Y")

for row in rows_yesterday:
    row.pop(0)
    row.insert(1,yesterday)
for row in rows_two_days_ago:
    row.pop(0)
    row.insert(1,two_days_ago)

total_covid = rows_two_days_ago + rows_yesterday 

In [322]:
for i in total_covid:
    if len(i) > 16:
        print(i)

In [323]:
total_covid_df = pd.DataFrame(total_covid, columns=header)
total_covid_df.to_csv('worldmeter_covid19.csv',sep=',',index=False)

In [324]:
total_covid_df = pd.read_csv('worldmeter_covid19.csv')

In [325]:
def convert_numeric(num):
    condition = {",":"","+":""}
    if num == "nan" or num == " " or num == "  " or num == " N/A":
        return 0

    for i, j in condition.items():
        num = num.replace(i, j)
    return int(num)
for i in range(2,15):
    total_covid_df[header[i]] = total_covid_df[header[i]].apply(lambda x:convert_numeric(str(x)))
    
total_covid_df[header[15]] = total_covid_df[header[15]].apply(lambda x:str(x).replace(" ",""))
total_covid_df['Date'] = pd.to_datetime(total_covid_df['Date'],format="%d-%m-%Y")

In [326]:
total_covid_df

,Country,Date,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent
0,USA,2022-02-27,80614425,33032,974440,562,53221683,173485,26418302,8279,241201,2916,949062252,2839628,334220623,NorthAmerica
1,India,2022-02-27,42924130,8013,513843,87,42307686,16765,102601,8944,30606,366,766757518,546710,1402495010,Asia
2,Brazil,2022-02-27,28768104,24054,649195,206,26183623,101112,1935286,8318,133765,3019,63776166,296545,215064353,SouthAmerica
3,France,2022-02-27,22689332,42600,138135,76,20666313,45900,1884884,2484,346333,2109,246629975,3764597,65512990,Europe
4,UK,2022-02-27,18859389,27312,161315,46,17350941,98844,1347133,289,275411,2356,480341745,7014619,68477240,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,Western Sahara,2022-02-28,10,0,1,0,8,0,1,0,16,2,0,0,621662,Africa
448,MS Zaandam,2022-02-28,9,0,2,0,7,0,0,0,0,0,0,0,0,nan
449,Marshall Islands,2022-02-28,7,0,0,0,7,0,0,0,117,0,0,0,59852,Australia/Oceania
450,Saint Helena,2022-02-28,2,0,0,0,2,0,0,0,327,0,0,0,6107,Africa
